In [6]:
import pandas as pd
import pickle
import numpy as np

In [15]:
df_ivs = pickle.load(open('data/ivs_exploded_100.p', 'rb'))
df_ivs_old = pickle.load(open('data/ivs_exploded_100_old.p', 'rb'))
routes_dests = pickle.load(open('data/users_ship_specific_routes.p', 'rb'))
df_ships = pd.read_excel('data/ship_types.xlsx')

G = pickle.load(open("data/network.p", "rb"))
paths = pickle.load(open("data/paths.p", "rb"))
df_random = pickle.load(open("data/df_random.p", "rb"))
df_abm = pickle.load(open("data/df_abm.p", "rb"))
non_zero_flows = pickle.load(open("data/non_zero_flows.p", "rb"))

In [16]:
df_random

,origin,destination,trip_count,hour,M12,M8,BII-6b,M10,BIIa-1,M9,...,M0,C2l,BII-2L,B02,C1b,C2b,B01,C1l,route_v,key
93,NLRTM,NLAMS,181,1,0,110,0,0,0,61,...,0,0,0,0,0,0,0,0,2,"(NLRTM, NLAMS, 2)"
94,NLRTM,NLAMS,106,1,64,0,0,15,0,0,...,0,0,0,0,0,0,0,0,2,"(NLRTM, NLAMS, 2)"
95,NLRTM,NLAMS,71,1,0,58,0,0,0,4,...,0,0,0,0,0,0,0,0,2,"(NLRTM, NLAMS, 2)"
96,NLRTM,NLAMS,19,1,1,0,0,2,0,0,...,0,0,0,0,0,0,0,0,2,"(NLRTM, NLAMS, 2)"
97,NLRTM,NLAMS,52,1,8,9,0,4,0,17,...,0,0,0,0,0,0,0,0,2,"(NLRTM, NLAMS, 2)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2840,NLTNZ,NLVLI,1,23,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"(NLTNZ, NLVLI, 0)"
2841,NLTNZ,NLVLI,1,23,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"(NLTNZ, NLVLI, 0)"
2842,NLTNZ,NLVLI,24,23,0,24,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"(NLTNZ, NLVLI, 0)"
2843,NLTNZ,NLVLI,3,23,0,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,"(NLTNZ, NLVLI, 0)"


In [22]:
type_list = list(df_random.iloc[:, 4:-2])
for hour in range(24):
    df_1 = df_random.loc[(df_random.hour == hour)]
    for timestep in range(60):
        for harbour in list(set(list(df_1.origin.unique()) + list(df_1.destination.unique()))):
            a = df_1.loc[(df_1.origin==harbour)|(df_1.destination == harbour)]
            for i,j in enumerate(a.index):
                # chance that a vessel is generated is equal to 1/2 (either spawn at origin or dest) * trip_count/365
                # (because trip_count is yearly) *(time_step/hours)
                if 0.5 * (df_1.trip_count[j]/365) * (1/60) >= np.random.random():
                    # determine vessel type
                    prob = list(a.iloc[i, 4:-2].values / a.iloc[i, 4:-2].sum())
                    to_pick = type_list
                    ship_type = np.random.choice(a=to_pick, size=1, replace=True, p=prob)
                    # randomly draw ship types for each OD pair
                    print(ship_type, "Vessel departed at", df_1.origin[j], hour, ':', timestep, "heading to", df_1.destination[j], "via route", df_1.key[j])


['M8'] Vessel departed at NLRTM 1 : 14 heading to NLAMS via route ('NLRTM', 'NLAMS', 2)
['M8'] Vessel departed at NLRTM 1 : 16 heading to NLAMS via route ('NLRTM', 'NLAMS', 2)
['M8'] Vessel departed at NLRTM 1 : 22 heading to NLAMS via route ('NLRTM', 'NLAMS', 2)
['M10'] Vessel departed at NLRTM 1 : 56 heading to NLAMS via route ('NLRTM', 'NLAMS', 2)
['M8'] Vessel departed at NLRTM 2 : 13 heading to NLMOE via route ('NLRTM', 'NLMOE', 0)
['M8'] Vessel departed at NLRTM 2 : 37 heading to NLMOE via route ('NLRTM', 'NLMOE', 0)
['M9'] Vessel departed at NLRTM 2 : 45 heading to NLMOE via route ('NLRTM', 'NLMOE', 0)
['M8'] Vessel departed at NLRTM 2 : 53 heading to NLMOE via route ('NLRTM', 'NLMOE', 0)
['M8'] Vessel departed at NLRTM 2 : 55 heading to NLMOE via route ('NLRTM', 'NLMOE', 0)
['M8'] Vessel departed at NLRTM 3 : 5 heading to NLMOE via route ('NLRTM', 'NLMOE', 0)
['BIIL-1'] Vessel departed at NLRTM 3 : 10 heading to NLMOE via route ('NLRTM', 'NLMOE', 0)
['M4'] Vessel departed at NL

In [18]:
type_list

['M12',
 'M8',
 'BII-6b',
 'M10',
 'BIIa-1',
 'M9',
 'BII-6l',
 'C3b',
 'BII-4',
 'M7',
 'M6',
 'BIIL-1',
 'C3l',
 'M5',
 'M11',
 'BI',
 'M3',
 'M2',
 'M1',
 'BII-1',
 'BII-2b',
 'M4',
 'B03',
 'C4',
 'B04',
 'M0',
 'C2l',
 'BII-2L',
 'B02',
 'C1b',
 'C2b',
 'B01',
 'C1l']

In [19]:
def random_vessel_generator(df_prob, load=1):
    """ Function to generate random vessel data.
    Parameters
    ----------
    df_prob: pd.DataFrame
    This dataframe must contain historical travel data
    load: float
    This float is equal to the simulated occupation of the network and may be read as the part of the traffic that is
    electric.
    """
    df_prob = df_prob.loc[df_prob.trip_count != 0]
    df_prob.reset_index(inplace=True, drop=True)
    df_prob = df_prob.fillna(0)

    # create dict to store random prob based values later on
    main_dict = {i: [] for i in df_prob.columns}

    # copy origin and destination from original df
    main_dict['origin'] = []
    main_dict['destination'] = []
    main_dict['route_v'] = []
    main_dict['hour'] = []
    main_dict['trip_count'] = []

    # fill nan values and create type list
    type_list = list(df_prob.iloc[:, 4:-1])
    for i in range(24):
        df_temp = df_prob.loc[df_prob.hour == i]
        df_temp = df_temp.reset_index(drop=True)
        total = df_temp.trip_count.sum()
        prob_r = [i / total for i in df_temp.trip_count]
        to_pick_r = list(np.arange(0, len(df_temp)))
        count_r = total * 1
        # randomly draw ODs
        rand_routes = np.random.choice(a=to_pick_r, size=round(count_r), replace=True, p=prob_r)
        unique_r, counts_r = np.unique(rand_routes, return_counts=True)
        temp_type_dict_r = dict(zip(unique_r, counts_r))

        for pair in df_temp.index:
            # store trip count data previous draw
            if pair in temp_type_dict_r.keys():
                main_dict['trip_count'].append(temp_type_dict_r[pair])
                main_dict['origin'].append(df_temp.origin[i])
                main_dict['destination'].append(df_temp.destination[i])
                main_dict['route_v'].append(df_temp.route_v[i])
                main_dict['hour'].append(i)
                # print(main_dict['trip_count'])
                # determine parameters to randomly draw ship types for each OD pair
                prob = list(df_temp.iloc[pair, 4:-1].values / df_temp.iloc[pair, 4:-1].sum())
                to_pick = type_list
                # goes wrong now because may be zero ...
                count = temp_type_dict_r[pair]

                # randomly draw ship types for each OD pair
                rand_vessels = np.random.choice(a=to_pick, size=round(count), replace=True, p=prob)
                unique, counts = np.unique(rand_vessels, return_counts=True)
                temp_type_dict = dict(zip(unique, counts))
                for key in type_list:
                    if key in temp_type_dict.keys():
                        main_dict[key].append(temp_type_dict[key])
                    else:
                        main_dict[key].append(0)

    df_return = pd.DataFrame.from_dict(main_dict)

    return df_return

In [4]:
a1 = random_vessel_generator(df_ivs, 1)

In [5]:
a1.insert(3, 'actual', list(a1.iloc[:, 4:-1].sum(axis=1)))
a1.loc[a1.trip_count != a1.actual]

,origin,destination,trip_count,actual,hour,M12,M8,BII-6b,M10,BIIa-1,...,B04,M0,C2l,BII-2L,B02,C1b,C2b,B01,C1l,route_v


In [93]:
a1

,origin,destination,trip_count,actual,hour,M12,M8,BII-6b,M10,BIIa-1,...,B04,M0,C2l,BII-2L,B02,C1b,C2b,B01,C1l,route_v
0,NLRTM,NLAMS,155,155,0,0,94,0,0,2,...,0,0,0,0,0,0,0,0,0,1
1,NLRTM,NLAMS,161,161,0,98,0,0,14,0,...,0,0,0,3,0,0,0,0,0,1
2,NLRTM,NLAMS,55,55,0,0,39,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,NLRTM,NLAMS,25,25,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,NLRTM,NLAMS,80,80,0,9,20,0,3,1,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2850,NLTNZ,NLVLI,5,5,23,0,5,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2851,NLTNZ,NLVLI,3,3,23,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2852,NLTNZ,NLVLI,19,19,23,0,16,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2853,NLTNZ,NLVLI,3,3,23,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
